In [166]:
# Cargar bases de datos de carpeta data
import pandas as pd
import numpy as np

# Base de datos
df1 = pd.read_csv("data/BD_Asegurados_Expuestos.csv", sep=";")
df2 = pd.read_csv("data/BD_Diagnostico.csv", sep=";")
df3 = pd.read_csv("data/BD_Genero.csv", sep=";")
df4 = pd.read_csv("data/BD_Reclamaciones.csv", sep=";")
df5 = pd.read_csv("data/BD_Regional.csv", sep=";")
df6 = pd.read_csv("data/BD_SocioDemograficas.csv", sep=";", low_memory=False)
df7 = pd.read_csv("data/BD_UtilizacionesMedicas.csv", sep=";")
fechas = pd.read_csv("data/fechas.csv", sep=";")

# Unión bases de datos

Procesamiento de fechas
1. Se utiliza la funcion UNICOS en cada libro de excel
2. Se pasan esos valores a un libro de excel llamado fechas en una sola columna
3. Se vuelve a realizar la función UNICOS para obtener los únicos de todos los libros
4. Se realiza un merge de bases de datos en python para obtener los valores de las fechas

In [167]:
################################ ASIGNACIÓN DE FECHAS CON BASE FECHAS ##################################
# Fecha de inicio, cancelación y fin
df1 = (
    pd.merge(df1, fechas, left_on="FECHA_INICIO", right_on="cod_fecha", how="left")
    .merge(fechas, left_on="FECHA_CANCELACION", right_on="cod_fecha", how="left")
    .merge(fechas, left_on="FECHA_FIN", right_on="cod_fecha", how="left")
    .drop(
        columns=[
            "cod_fecha",
            "FECHA_INICIO",
            "FECHA_FIN",
            "FECHA_CANCELACION",
            "cod_fecha",
            "cod_fecha_x",
            "cod_fecha_y",
        ]
    )
    .rename(
        columns={
            "fecha_x": "FECHA_INICIO",
            "fecha_y": "FECHA_CANCELACION",
            "fecha": "FECHA_FIN",
        }
    )
)

# Fecha de nacimiento
df6 = (
    pd.merge(df6, fechas, left_on="FechaNacimiento", right_on="cod_fecha", how="left")
    .drop(columns=["cod_fecha", "FechaNacimiento"])
    .rename(columns={"fecha": "FechaNacimiento"})
)

# Fecha de reclamación
df7 = (
    pd.merge(df7, fechas, left_on="Fecha_Reclamacion", right_on="cod_fecha", how="left")
    .drop(columns=["cod_fecha", "Fecha_Reclamacion"])
    .rename(columns={"fecha": "Fecha_Reclamacion"})
)

###############################  Convertir a datetime las columnas de fechas ###############################
df1["FECHA_INICIO"] = pd.to_datetime(df1["FECHA_INICIO"], format="%d/%m/%Y")
df1["FECHA_CANCELACION"] = pd.to_datetime(df1["FECHA_CANCELACION"], format="%d/%m/%Y")
df1["FECHA_FIN"] = pd.to_datetime(df1["FECHA_FIN"], format="%d/%m/%Y")
df6["FechaNacimiento"] = pd.to_datetime(df6["FechaNacimiento"], format="%d/%m/%Y")
df7["Fecha_Reclamacion"] = pd.to_datetime(df7["Fecha_Reclamacion"], format="%d/%m/%Y")

In [168]:
######################################### UNION DE BASES DE DATOS #########################################

######################## df7 = Union de bases Diagnostico y reclamación ##############################

df7 = pd.merge(df7, df2, on="Diagnostico_Codigo", how="left").merge(
    df4, on="Reclamacion_Cd", how="left"
)

####################### Procesamiento para merge Regional_Id ##############################
# Cambiar nombre de Regional a Regional_Id de df6
df6 = df6.rename(columns={"Regional": "Regional_Id"})

# Reemplazar de Regional_Id de df6 el #N/D por -1
df6["Regional_Id"] = df6["Regional_Id"].replace("#N/D", -1)

# Convertir Regional_Id de df6 a object
df6["Regional_Id"] = df6["Regional_Id"].astype("int64")

####################### df6 = Union de bases Sexo, Regional ##############################
df6 = pd.merge(df6, df3, on="Sexo_Cd", how="left").merge(
    df5, on="Regional_Id", how="left"
)


####################### Unión final ##############################
df1 = df1.rename(columns={"Asegurado_Id": "Afiliado_Id"})

# Union de df1, df6 y df7 con Asegurado_Id
df = pd.merge(df7, df6, on="Afiliado_Id", how="outer").merge(
    df1, on="Afiliado_Id", how="outer"
)

df.shape

(894056, 22)

# Limpieza de datos

In [169]:
######################################### TRATAMIENTOS MENORES #########################################
# Cambiar Sexo_Cd.1 por Sexo
df = df.rename(columns={"Sexo_Cd.1": "Sexo"})

# cambiar nombre columna Valor_Utilizaciones por Costo_Total_Utilizaciones
df = df.rename(columns={"Valor_Utilizaciones": "Costo_Total_Utilizaciones"})


######################################### LIMPIEZA #########################################
# quitar categoría "Sin información" de Sexo (5 datos)
df = df[df["Sexo"] != "Sin Informacion"]

# Quitar categoría "Sin información" de Regional_desc (acumulado 110)
df = df[df["Regional_desc"] != "Sin Información"]

# Seleccionar solo los registros del 2019 (acumulado 70599)
df = df[df["Fecha_Reclamacion"].dt.year == 2019]


######################################### CREACIÓN DE COLUMNAS #########################################
# Crear columna con FechaNacimiento para edad
df["Edad"] = df["Fecha_Reclamacion"].dt.year - df["FechaNacimiento"].dt.year
# Se eliminan las filas con Edad negativa (acumulado 73236)
df = df[df["Edad"] >= 0]

# Las Numero_Utilizaciones = 0 se eliminan
df = df[df["Numero_Utilizaciones"] > 0]

# Columna con Valor_Utilizaciones / Numero_Utilizaciones
df["Precio_Utilización"] = df["Costo_Total_Utilizaciones"] / df["Numero_Utilizaciones"]


# Cambiar nombre de columnas Numero_Utilizaciones por Cantidad_Utilizaciones y Valor_Utilizaciones por Costo_Total_Utilización
df = df.rename(
    columns={
        "Numero_Utilizaciones": "Cantidad_Utilizaciones",
    }
)

# Convertir variable Cantidad_Utilizaciones a int64
df["Cantidad_Utilizaciones"] = df["Cantidad_Utilizaciones"].astype("int64")

# Crear rango de edades así: 0-1, 1-12, 13-18, 19-29, 30-59, 60-69, 70-79, 80-89, 90-99, 100+
df["Edad"] = df["Edad"].astype("int64")
df["rango_edad"] = pd.cut(
    df["Edad"],
    bins=[-1, 1, 12, 18, 29, 59, 69, 79, 89, 99, 200],
    labels=[
        "0-1",
        "1-12",
        "13-18",
        "19-29",
        "30-59",
        "60-69",
        "70-79",
        "80-89",
        "90-99",
        "100+",
    ],
)


# Días_en_reclamar = FECHA_FIN - FECHA_INICIO en días redondeando a 0 decimales y quitando los valores negativos
df["Días_en_reclamar"] = (
    ((df["Fecha_Reclamacion"] - df["FECHA_INICIO"]).dt.days)
).round()


# Acá se quitan los valores negativos, no tiene sentido que se reclame antes de que inicie la póliza (acumulado 255989)
df = df[df["Días_en_reclamar"] >= 0]

################################## Columnas a eliminar ##################################

df = df.drop(
    columns=[
        "Reclamacion_Cd",  # Se tomó el valor y no el código
        "Diagnostico_Codigo",  # Se tomó el valor y no el código
        "Sexo_Cd",  # Se tomó el valor y no el código
        "Regional_Id",  # Se tomó el valor y no el código
        "FechaNacimiento",  # Se tomó la edad
        "FECHA_CANCELACION",  # Se tomó la fecha de fin
        "Poliza_Id",  # No se utilizará
    ]
)

df.shape

(637770, 19)

ANTES DE LA LIMPIEZA DE DATOS:
* En un inicio tenemos 894056 datos con 22 columnas

TRATAMIENTOS MENORES
* Cambiar el nombre de la variable Sexo_Cd.1 por Sexo
* Cambiar el nombre de la columna Valor_Utilizaciones por Costo_Total_Utilizaciones

LIMPIEZA DE DATOS
* Quitamos la categoría "Sin Información" de la variable Regional_desc (110 datos) y de la variable Sexo (5 datos)
* Nos centraremos en los afiliados que compraron la póliza en septiembre de 2019 y la reclamaron después de esa fecha. Además tomamos esta decisión porque el número de afiliados que no son del 2019 es muy bajo (10%), con lo que tendríamos el 90% para realizar el análisis y es algo de lo que podríamos obtener información relevante
* Se quitaron las edades menores a 0
* La cantidad de utilizaciones que son 0 se eliminan porque no tienen sentido y son solo 254 registros
* Los días a reclamar menores a 0 se eliminaron porque nos enfocamos en los que reclamaron después de la compra de la póliza

CREACIÓN DE COLUMNAS
* Con la columna fecha de reclamación y fecha de nacimiento se crea una columna con la edad
* Creación columna Precio_Utilizacion = Costo_Total_Utilizaciones / Numero_Utilizaciones
* Días_en_reclamar = FECHA_FIN - FECHA_INICIO


ELIMINACIÓN DE COLUMNAS
Porque se tomó el valor y no el código
* Reclamacion_Cd 
* Diagnostico_Codigo
* Sexo_Cd
* Regional_Id


Porque se tomó otra columna 
* FechaNacimiento: Se tomó la edad
* FECHA_CANCELACION: Era la misma de FECHA_FIN
* Duración_Póliza: Se tomó el rango

La PolizaID podría ser algo muy influyente en nuestras variables objetivo para ver que póliza hace que el precio se incremente pero tiene 147 mil registros unicos y al volverlos dummies, se crearían demasiadas columnas por lo que se decide eliminar



Grupo etario: Nos basamos en lo siguiente para realizar la asignación del grupo etario:
* Reglamento de salud suplementaria expedido por la Superintendencia Nacional de Salud de Colombia
* Contratos de seguros de hospitalización y cirugía ofrecidos por las principales aseguradoras de Colombia

Haciendo énfasis es decir rangos más pequeños, de la vejez ya que son más propensos a tener enfermedades y a reclamar más.

* Menores de 1 año: 0-12 meses
* Niños de 1 a 12 años: 1-12 años
* Adolescentes de 13 a 18 años: 13-18 años
* Adultos jóvenes de 19 a 30 años: 19-29 años
* Adultos de 30 a 59 años: 30-59 años
* Adultos mayores de 60 años: 60-69 años
* Adultos mayores de 70 años: 70-79 años
* Adultos mayores de 80 años: 80-89 años
* Adultos mayores de 90 años: 90-99 años
* Adultos mayores de 100 años: 100 años o más

Finalizamos con 637770 registros con 19 columnas

# Tratamiento reclamaciones

Para reducir la cantidad de categorías en la variable tipo de reclamación, agrupamos algunas categorías en estos grandes grupos:
* TRATAMIENTO AMBULATORIO
* TRATAMIENTO MEDICO HOSPITALARIO
* TRATAMIENTO QUIRÚRGICO HOSPITALARIO
* CIRUGÍA AMBULATORIA


y juntamos 1 ANEXO EXEQUIAL con FUNERARIOS

In [170]:
# SI CONTIENE ESA PALABRA COLOCAR ESA PALABRA: CIRUGÍA AMBULATORIA
df.loc[
    df["Reclamacion_Desc"].str.contains("CIRUGIA AMBULATORIA"), "Reclamacion_Desc"
] = "CIRUGIA AMBULATORIA"
# SI CONTIENE ESA PALABRA COLOCAR ESA PALABRA: TRATAMIENTO QUIRURGICO HOSPITALARIO, TRATAMIENTO MEDICO HOSPITALARIO, TRATAMIENTOS AMBULATORIOS
df.loc[
    df["Reclamacion_Desc"].str.contains("TRATAMIENTO QUIRURGICO HOSPITALARIO"),
    "Reclamacion_Desc",
] = "TRATAMIENTO QUIRURGICO HOSPITALARIO"
df.loc[
    df["Reclamacion_Desc"].str.contains("TRATAMIENTO MEDICO HOSPITALARIO"),
    "Reclamacion_Desc",
] = "TRATAMIENTO MEDICO HOSPITALARIO"

# SI CONTIENE ESA PALABRA COLOCAR ESA PALABRA: TRATAMIENTOS MEDICO AMBULATORIOS & TRATAMIENTO MEDICO AMBULATORIO
df.loc[
    df["Reclamacion_Desc"].str.contains("TRATAMIENTOS MEDICO AMBULATORIOS"),
    "Reclamacion_Desc",
] = "TRATAMIENTO AMBULATORIO"
df.loc[
    df["Reclamacion_Desc"].str.contains("TRATAMIENTO MEDICO AMBULATORIO"),
    "Reclamacion_Desc",
] = "TRATAMIENTO AMBULATORIO"
df.loc[
    df["Reclamacion_Desc"].str.contains("TRATAMIENTO AMBULATORIO"), "Reclamacion_Desc"
] = "TRATAMIENTO AMBULATORIO"
df.loc[
    df["Reclamacion_Desc"].str.contains("TRATAMIENTOS AMBULATORIOS"),
    "Reclamacion_Desc",
] = "TRATAMIENTO AMBULATORIO"

# Cambiar ANEXO EXEQUIAL --> FUNERARIOS
df.loc[
    df["Reclamacion_Desc"].str.contains("ANEXO EXEQUIAL"), "Reclamacion_Desc"
] = "FUNERARIOS"

# Detalles mínimos

Convertimos las variables en tipo objeto y cambiamos el orden las columnas para mayor facilidad a la hora de interpretar

In [171]:
# conversión de variables categóricas y objeto
for col in [
    "Afiliado_Id",
    "Sexo",
    "rango_edad",
    "Regional_desc",
    "Diagnostico_Desc",
    "Reclamacion_Desc",
    "CANCER",
    "EPOC",
    "DIABETES",
    "HIPERTENSION",
    "ENF_CARDIOVASCULAR",
]:
    df[col] = df[col].astype("object")

################################## cambiar orden de las columnas ##################################

df = df[
    [
        "Afiliado_Id",
        "Sexo",
        "Edad",
        "rango_edad",
        "Regional_desc",
        "Fecha_Reclamacion",
        "FECHA_INICIO",
        "FECHA_FIN",
        "Días_en_reclamar",
        "Cantidad_Utilizaciones",
        "Precio_Utilización",
        "Costo_Total_Utilizaciones",
        "Diagnostico_Desc",
        "Reclamacion_Desc",
        "CANCER",
        "EPOC",
        "DIABETES",
        "HIPERTENSION",
        "ENF_CARDIOVASCULAR",
    ]
]

## Tratamiento datos nulos

In [172]:
# Nulos de df porcentaje
print(df.shape)
print(df.isnull().sum() / len(df) * 100)

(637770, 19)
Afiliado_Id                  0.0
Sexo                         0.0
Edad                         0.0
rango_edad                   0.0
Regional_desc                0.0
Fecha_Reclamacion            0.0
FECHA_INICIO                 0.0
FECHA_FIN                    0.0
Días_en_reclamar             0.0
Cantidad_Utilizaciones       0.0
Precio_Utilización           0.0
Costo_Total_Utilizaciones    0.0
Diagnostico_Desc             0.0
Reclamacion_Desc             0.0
CANCER                       0.0
EPOC                         0.0
DIABETES                     0.0
HIPERTENSION                 0.0
ENF_CARDIOVASCULAR           0.0
dtype: float64


# Guardar en formato pickle

In [173]:
# Guardar base de datos en pickle
df.to_pickle("data/df.pkl")

# Resumen

### **LIMPIEZA Y PREPARACIÓN DE DATOS**

#### 1. PROCESAMIENTO DE FECHAS
- Se reunieron y consolidaron valores únicos de fechas de reclamaciones desde múltiples fuentes mediante Excel y Python.

#### 2. LIMPIEZA INICIAL
- Se manejaron pequeños ajustes: renombrar columnas, eliminar categorías irrelevantes y centrarse en datos pertinentes.

#### 3. FILTRADO DE DATOS
- Se focalizó en afiliados que adquirieron pólizas en septiembre de 2019 y reclamaron después, representando el 90% de los datos para análisis relevante.
- Se eliminaron registros inválidos (edades negativas, utilizaciones de valor 0, días de reclamo negativos).

#### 4. CREACIÓN Y ELIMINACIÓN DE COLUMNAS
- Se crearon nuevas columnas como "Edad", "Precio_Utilizacion", y "Días_en_reclamar" para análisis detallado.
- Se eliminaron columnas innecesarias o redundantes.

#### 5. GRUPO ETARIO
- Se definió el grupo etario siguiendo estándares de salud y seguros, segmentando en rangos específicos.

#### 6. TRATAMIENTO DE TIPO DE RECLAMACIONES
- Se agruparon y simplificaron las categorías de "tipo de reclamación" para una mejor comprensión y análisis.

#### 7. DETALLES FINALES
- Se optimizó la presentación del dataset para facilitar su interpretación.

#### 8. CONTROL DE DATOS NULOS
- No se encontraron valores nulos en el dataset.

---

Este proceso de limpieza y preparación de datos permitió organizar la información de manera coherente para un análisis más efectivo.
